In [ ]:
# Baseline Model on the Sonar Dataset
import keras
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout,PReLU, LeakyReLU, Embedding, LSTM, Convolution1D
from keras.initializers import lecun_uniform, Constant
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
import sys
# Local toolset
import predictive
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.layers import Input, Lambda
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding, Flatten, Dense
from keras.initializers import lecun_uniform, Constant, glorot_normal, he_normal, glorot_uniform, he_uniform
from keras.callbacks import ModelCheckpoint
from keras import regularizers
import gc
from keras import backend as K_s
import json
from sklearn.preprocessing import MinMaxScaler
import secrets, copy
from numpy import linalg as LA
#from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.utils import class_weight


import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras

In [ ]:
def Metrics(model,X_val,Y_val,X_val_0,Y_val_0,X_val_1,Y_val_1):
    
    metrics=[]
    

    tmp=model.evaluate(X_val_0, Y_val_0, batch_size=len(Y_val_0), verbose=0)
    
    acc_0=tmp[1]
    loss_0=tmp[0]


    tmp=model.evaluate(X_val_1, Y_val_1, batch_size=Y_val_1, verbose=0)

    acc_1=tmp[1]
    loss_1=tmp[0]


    tmp=(loss_0+loss_1)/2
    
    metrics.append(tmp)
    
    tmp=(acc_0+acc_1)/2
    
    metrics.append(tmp)
    
    
    pred=model.predict(X_val,batch_size=len(X_val))
    
    pred = pred.reshape(Y_val.shape)
    
    stats = predictive.stats.evaluate(predictive.stats.DataKind.PANDAS, pred, Y_val, .5, sys.stdout)
    
    metrics.append(stats)
        
    return metrics 

In [ ]:
def create_model():
    K_s.clear_session()
    gc.collect()
    
    input_dim=24428
    
    model = Sequential()
    model.add(Dense(200,input_dim=input_dim, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    
    return model

In [ ]:
model = create_model()

epochs=300


X_train= np.load("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Data/X_train.npy").astype(np.float16)
Y_train= np.load("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Data/Y_train.npy").astype(np.float16)


X_val= np.load("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Data/X_test.npy").astype(np.float16)
Y_val= np.load("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Data/Y_test.npy").astype(np.float16)


X_val_0=copy.deepcopy(X_val[Y_val[:]==0])
Y_val_0=copy.deepcopy(Y_val[Y_val[:]==0])


X_val_1=copy.deepcopy(X_val[Y_val[:]==1])
Y_val_1=copy.deepcopy(Y_val[Y_val[:]==1])


class_weights = class_weight.compute_class_weight('balanced', np.unique(Y_train),Y_train)

class_weights = dict(enumerate(class_weights))


res_suivie=[]

model.save("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Models/model_record_DP.h5")


for epoch in range(epochs):
    
    
    model=load_model("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Models/model_record_DP.h5")
    
    
    print("epoch:",epoch)
    
    model.fit(X_train, Y_train, epochs=1,batch_size=100,verbose=1,class_weight= class_weights)
    
    indices=np.random.choice(np.arange(len(X_train)),len(X_train),replace=False)
    
    X_train=X_train[indices]
    Y_train=Y_train[indices]
    
    
    model.save("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Models/model_record_DP.h5")
    
    
    res_suivie.append([Metrics(model,X_val,Y_val,X_val_0,Y_val_0,X_val_1,Y_val_1)])
    
    with open("/notebooks/Raouf/MAGNET_Lille/Project_Dream/KDD_experiments/Results/Centralized_"+str(epoch)+".json", 'w') as f:
         json.dump(res_suivie, f)
            
    K_s.clear_session()
    del model
    gc.collect()